In [1]:
import pandas as pd
import numpy as np

import traceback
import psycopg2

from datetime import datetime
from datetime import timedelta
import os
import re 

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [3]:
date = datetime.now().strftime('%d-%m-%Y')

# National Life

## Functions

Función para el separado de los nombres en las columnas Menber First Name y Merber Last Name. Separa los nombres de los apellidos

In [11]:
def split_names(name:str, first:bool=True, split_by=' ') -> str:
    name = name.split(split_by, maxsplit=1)
    if first:
        return name[0].replace(',','')
    return name[1].replace(',','')

Homologa los status del carrier a los que se usan en el BOB

In [12]:
def replace_status(x):
    try:
        if 'Pending EFT -' in x:
            x = x.split('-')[0].strip()
            
        replaced_status = df_status[df_status['Status Carrier'] == x]['Status SF'].values[0]
        return replaced_status
    except IndexError:
        return None

In [18]:
def extractCarrier(df, carrier_name, carrier_column):
    df =  df[df[carrier_column] == carrier_name]
    return df

### Conection

In [41]:
def start():
    connection = psycopg2.connect(dbname = 'Carriers',
                    host = '188.166.63.213',
                    port = '5432',
                    user = 'cperez',
                    password = 'srb4o7SlqAIf')

    return connection

Función que se usa para hacer queries a la base de datos

In [42]:
def query_db (query):
    try:
        connection = start()
        with connection.cursor() as cursor:
            cursor.execute(query)
            result = cursor.fetchall()
        connection.commit()
        return result
    except Exception as ex:
        traceback.print_exc(ex)
    finally:
        connection.close()

        

In [43]:
connection = start()

### Carga

### BOB

In [87]:
df_b = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/bob/report1674816107684.csv', encoding='latin-1', low_memory=False)
df_b.head(3)

,Book of Business: BOB Name,Account Executive Name,Carrier,NPN AOR,Agent AOR,Internal Agent,NPN Internal Agent,Contact,Policy Number,Source,...,Email,Metal Level,Zip Code,Book of Business: Last Modified Date,Note,Claro Type,Previous BOB ID,Carrier State line of Business,Book of Business: ID,Account Executive
0,BOB# 00000001,Miguel Bautista,Molina,1647487.0,Amarilys Perez Lazo,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192951620161,Direct,...,NaN,Silver,NaN,12/20/2022,NaN,Renewals,NaN,Molina Florida,a1C1G000007eLyj,
1,BOB# 00000002,Miguel Bautista,Molina,1647487.0,Amarilys Perez Lazo,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192921397071,Direct,...,NaN,Bronze,NaN,12/20/2022,NaN,Renewals,NaN,Molina Florida,a1C1G000007eLyk,
2,BOB# 00000003,Miguel Bautista,Molina,1647487.0,AMARILYS PEREZ LAZO,Amarilys Perez Lazo,1647487.0,Amarilys Perez Lazo,FLE192921385481,Direct,...,NaN,Bronze,NaN,12/20/2022,V-ActualizaciÃ³n 09-01,Renewals,NaN,Molina Florida,a1C1G000007eLyl,


In [15]:
df_a = (pd.read_excel('//mnt/c/Users/cesar/Desktop/claro/life/NationalLife/NLG_AllNewBusinessReport_01262023.xlsx', header=5)
                     .dropna(axis='columns', how='all')
                     .dropna( how='all'))

df_a = df_a.iloc[:-3,:]

In [16]:
df_a.head()

,Policy #,Insured / Annuitant,Agent,Status,Delivery,Action,Submitted,Modal Premium,Product,Sent,Owner,Submit Method,Case Manager,AAP*,Agent #,Agency,Company Code
0,LS759986500,Juan Carlos Marquez,VERA MARIA EUGENIA,PENDING,-,[01/25/2023] E-INSPECTION REPORT\n[01/25/2023]...,01/25/2023,6975,LSW Level Term 30-G,NaN,Juan Carlos Marquez,e-APP,Javier Latigo,6975,4330R,INFINITY MARKETING - 517,LS
1,LS760016000,Elena Nunez,XIXLOE PINEDA CLAUDIA,PENDING,-,[01/25/2023] DRAFT 1ST PREMIUM,01/25/2023,76.56,LSW Level Term 30-G,NaN,Elena Nunez,e-APP,Amanda Myers,870,5105U,INFINITY MARKETING - 517,LS
2,LS759991700,"Rodriguezcampos, Pedro",VARGAS ESTHER,APPROVED,-,[01/24/2023] DRAFT 1ST PREMIUM\n[01/24/2023] V...,01/24/2023,113.08,LSW Level Term 30-G,NaN,Pedro Rodriguezcampos,e-APP,Holly Clark,1285,6197T,INFINITY MARKETING - 517,LS
3,LS141354800,"Butler, Kaiden L",BOYKIN ANGELA,PENDING,-,NaN,01/23/2023,60,FlexLife,NaN,Angela Boykin,e-APP,Amanda Myers,149.5,6137U,INFINITY MARKETING - 517,LS
4,LS759930200,"Romero, Jose",RAMIREZ SUQUINO DUNIA,"Issued, Pending EFT - 02/04/2023",eDelivery Completed,NaN,01/23/2023,40.22,LSW Level Term 30-G,01/25/2023,Jose Romero,e-APP,Holly Clark,457,5039U,INFINITY MARKETING - 517,LS


## Homologación

In [17]:
df_a.columns

Index(['Policy #', 'Insured / Annuitant', 'Agent', 'Status', 'Delivery',
       'Action', 'Submitted', 'Modal Premium', 'Product', 'Sent', 'Owner',
       'Submit Method', 'Case Manager', 'AAP*', 'Agent #', 'Agency',
       'Company Code'],
      dtype='object')

In [18]:
df_a = df_a.rename(
    columns={
        'Policy #':'PolicyNumber',
        'Insured / Annuitant':'Insured',
        'Agent':'Agent AOR',
        'Status':'Status',
        'Submitted':'EffectiveDate',
        'AAP*': 'FinalPremium',
        'Agent #':'AgentNumber'
        
        
    }
)

### Limpiar Policy Number

Queitar los prefijos a los policy numbers del carriers

In [19]:
def limpiarPolicyNumber(df, column_name, replace_string:str='LS'):
    df = df[column_name].str.replace(replace_string,'')
    return df

In [20]:
df_a['PolicyNumberClean'] = limpiarPolicyNumber(df_a, 'PolicyNumber')

### Extraer las polizas de National Life

Extraer carrier

In [22]:
df_b = extractCarrier(df_b, 'National Life', 'Carrier')

### Hacer join

In [23]:
def leftJoinBOB(new_policies, bob_policies):
    upload = list(set(new_policies) - set(bob_policies) )
    return upload

In [24]:
upload_a = leftJoinBOB(df_a['PolicyNumberClean'].values, df_b['Policy Number'].values)

In [25]:
upload_b = leftJoinBOB(df_a['PolicyNumber'].values, df_b['Policy Number'].values)

In [26]:
upload = set(upload_a + [i.replace('LS', '') for i in upload_b])

In [27]:
df_upload = df_a[df_a['PolicyNumberClean'].isin(upload)]

### Upload into template

In [28]:
df_upload.reset_index(drop=True, inplace=True)

In [29]:
df_template = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/Format_Create_BOB.csv').head(0)

In [30]:
df_template

,Carrier___,LOB____,Carrier,Agent AOR,NPN AOR,Contact____,NPN_Internal____,Contact,Policy Number,Source,...,Subsidy,Annual Premium,Paid Thru Date,Days Past Due,Phone Number,Email,Metal Level,Zip Code,Claro Type,Carrier State line of Business


In [31]:
df_template['Member First Name'] = df_upload['Insured'].apply(lambda x : split_names(x, first=False))

In [32]:
df_template['Member Last Name'] = df_upload['Insured'].apply(lambda x : split_names(x))

In [33]:
df_template.reset_index(drop=True, inplace=True)

In [34]:
df_template['Policy Number'] = df_upload['PolicyNumberClean']

In [35]:
df_template['Final Premium'] = df_upload['FinalPremium']

In [36]:
df_template['Plan'] = df_upload['Product']

In [37]:
df_template['Agent AOR'] = df_upload['Agent AOR']

In [38]:
df_template['Effective Date'] = df_upload['EffectiveDate']

In [39]:
df_template['LOB____'] = 'Life'

#### Source

Dependiendo si el npn se encuentra en la lista de npn´s referenciales, el source va a ser directo (En caso de que no se encuentre), o referencial

In [40]:
def assign_source(referal_agents:list, df_template, column_agent_numbers:str, df_upload) -> pd.DataFrame:
    df_s = df_template.copy()
    mask_s = df_upload[column_agent_numbers].isin(referal_agents)
    
    df_s.loc[mask_s,  'Source'] = 'Referal'
    df_s.loc[~mask_s, 'Source'] = 'Direct'
    
    return df_s

In [41]:
def getSource(query,connection, df_template, df_upload, column_agent_numbers='AgentNumber') -> pd.DataFrame:
    referal_agents = [i[0] for i in query_db(query)]
    referal_list_agents = list(set(referal_agents).intersection(set(df_upload[column_agent_numbers].values)))
    
    df = assign_source(referal_list_agents, df_template, column_agent_numbers, df_upload)
    return df

In [42]:
npn_referals = [18233171, 7388594, 19603471, 18522178, 19106367, 16001347, 10041154, 20462802, 17686050]

npn_referals = str([str(i)  for i in npn_referals]).replace('[', '').replace(']','')


In [43]:
query_referals_agents = """
    SELECT agent_writing_number
    FROM agent_contract 
    WHERE npn in ({list_refarals}) ;
""".format(list_refarals=npn_referals)

In [44]:
df_template = getSource(query_referals_agents, connection, df_template, df_upload)

#### Mermbers

Siempre es uno

In [45]:
df_template['Members'] = 1

#### State

Siempre es Florida

In [46]:
df_template['State'] = 'FL'

#### Record Type

Si la poliza es de Jose m Parra, el record types es Direct Sales, de lo contrario es BOB

In [47]:
def recordType(df_upload, df_template, column_name='Record Type Id', column_agent='AgentNumber'):
    
    record_types = dict(query_db(query_record_type))
    agent_writing_number = query_db(query_awn_jose_m)
    agent_writing_number_jose_m = [i[0] for i in agent_writing_number]
    
#     jose_m_parra_awn_in_df = list(set(agent_writing_number).intersection(set(df_upload[column_agent].values)))
    others_awn = list(set(df_upload[column_agent].values) - set(agent_writing_number_jose_m))
    
    df_r = df_template.copy()
    mask = df_upload[column_agent].isin(others_awn)
    df_r.loc[mask, column_name] = record_types['BOB']
    df_r.loc[~mask, column_name] = record_types['Direct Sales']
    
    return df_r

In [48]:
query_record_type = """
    SELECT record_type, id  FROM record_type rt ;
"""

In [ ]:
query_awn_jose_m = """
    SELECT agent_writing_number
    FROM agent_contract 
    WHERE npn = '17795437'
    ;
"""

In [50]:
df_template = recordType(df_upload, df_template)

#### Carrier

In [51]:
query_carrier_id = """
    SELECT carrier_id  
    FROM carrier c 
    WHERE carrier = '{carrier_name}' and state_c = '{carrier_state}'
    ;
""".format(carrier_name= df_b['Carrier'].unique()[0], carrier_state=df_template['State'].unique()[0])

carrier_id = query_db(query_carrier_id)[0][0]

df_template['Carrier'] =  carrier_id

#### Status

Se homologa la columnas de carrier a las del BOB

In [52]:
df_status = pd.read_excel('/mnt/c/Users/cesar/Desktop/claro/life/Lista Status Life.xlsx')
df_status.head(3)

,Status Carrier,Status SF
0,Active,Active
1,APPROVED,Approved
2,Approved/Admin Review,Approved


In [53]:
df_template['Status'] = df_upload['Status'].apply(lambda x :replace_status(x) )

#### Policy Type

Si la poliza tiene más de 365 días y es renewal

In [54]:
def compare_years_policy(x): 
    one_year_ago = datetime.now() - timedelta(days=365)
    if one_year_ago >= x:
        return 'Renewal'
    return 'New Sale'

In [55]:
df_template['Policy Type'] = pd.to_datetime(df_upload['EffectiveDate']).agg(compare_years_policy)

#### Claro Type

In [56]:
df_template['Claro Type'] = 'New Sale'

#### Last Status

In [57]:
df_template['Last Status Modification Date'] = date

#### Loading Date

In [58]:
df_template['Loading Date'] = date

#### NPN AOR

De base de datos se extrae el npn del agente

In [59]:
query_agent_AOR = """
    SELECT contact, npn, agent_writing_number, id
    FROM agent_contract
    ;
"""

In [60]:
def extract_agent_aor(x, df_contact, column_name='NPN'):
    result = df_contact[df_contact['AgentWritingNumber'] == x][column_name].unique()
    assert len(result) < 2, 'Error Data Duplicada en el campo de la base de datos NPN'
    
    if len(result) > 0:
        return result[0]
    return None

In [61]:
def npnAOR(query_agent_AOR,connection, agent_number_colunm_name='AgentNumber'):
    list_contact = query_db(query_agent_AOR)
    df_contact = pd.DataFrame([{'Contact Name':i[0], 'NPN':i[1], 'AgentWritingNumber':i[2], 'Contact ID':i[3]} for i in list_contact])
    
    
    df_template['NPN AOR'] = df_upload[agent_number_colunm_name].apply(lambda x : extract_agent_aor(x, df_contact))
    df_template['Contact'] = df_upload[agent_number_colunm_name].apply(lambda x : extract_agent_aor(x, df_contact, 'Contact ID'))
    return df_template, df_contact

In [62]:
df_template, df_contact = npnAOR(query_agent_AOR, connection)

#### Contact ID

In [63]:
# def extract_contact_id(x, df_contact):
#     result = df_contact[df_contact['AgentWritingNumber'] == x]['Contact ID'].unique()
#     assert len(result) < 2, 'Error Data Duplicada en el campo de la base de datos NPN'
    
#     if len(result) > 0:
#         return result[0]
#     return None

In [64]:
# df_template['Contact ID'] = df_upload['AgentNumber'].apply(lambda x : extract_contact_id(x, df_contact))

#### Carrier State Line of Business

De base de datos de extrae el id del carrier state line of business

In [65]:
query_cslb = """
    select carrier_lob_id 
    from carrier c
    where carrier_id = '{carrier_id}' and state_c = '{state}';
"""

In [66]:
def making_query_cslb(df_template, query_cslb):
    carrier = df_template['Carrier'].unique()
    state = df_template.State.unique()
    
    assert (len(carrier) == 1) and (len(state) == 1), 'Error en la integridad de los datos.'
    
    query_cslb = query_cslb.format(carrier_id=carrier[0], state=state[0])
    value = query_db(query_cslb)[0][0]
    return value

In [67]:
def extract_cslb(df_template, query_cslb):
    cslb = making_query_cslb(df_template, query_cslb)
    df_template['Carrier State line of Business'] = cslb
    
    return df_template

In [68]:
df_template = extract_cslb(df_template, query_cslb)

## Time

In [69]:
df_template = df_template[['Agent AOR','NPN AOR', 'Contact', 'Carrier', 'State','Policy Number', 'Policy Type', 'Claro Type','Record Type Id', 'Carrier State line of Business','Source', 'Status', 'Effective Date',
            'Plan','Members', 'Last Status Modification Date', 'Loading Date',  'Final Premium','Member First Name', 'Member Last Name'
            ]]

In [70]:
df_times = df_template.copy()
df_times[['Last Status Modification Date', 'Loading Date']] = df_template[['Last Status Modification Date', 'Loading Date']].apply(lambda x : pd.to_datetime(x, format='%d-%m-%Y').dt.strftime('%m/%d/%Y'), axis='columns')
df_template = df_times

### Faltantes 

In [78]:
df_faltantes = df_template[df_template['Contact'].isna()]

In [79]:
df_faltantes

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name


In [73]:
def iterpolate(df, name, new_fields):
    df_t = df.copy()
    df_t.loc[df['Agent AOR'] == name, ['NPN AOR', 'Contact']] = new_fields
    return df_t

In [158]:
df_faltantes

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name


## Interpolacion

In [74]:
df_template = iterpolate(df_template, 'RAMIREZ SUQUINO DUNIA',  ('20544688','0031G00001YIab5QAD'))

In [77]:
df_template = iterpolate(df_template, 'COUTO ROSANGELA ROCHA',  ('18791909','0031G00001Jitp4QAB'))

In [94]:
#df_template = iterpolate(df_template, 'OROZCO ANGELA PATRICIA',  ('9709106','0031G00000vq9aHQAQ'))

In [95]:
#df_second_upload = df_template[df_template['Agent AOR'].isin(['COUTO ROSANGELA ROCHA', 'RIVERA JAIDIRA D', 'OROZCO ANGELA PATRICIA'])]

## Load

In [80]:
df_template.shape

(55, 20)

In [81]:
df_template.to_csv(f'/mnt/c/Users/cesar/Desktop/claro/imports/life_national_life{date}_.csv', index=False)

In [987]:
df_second_upload.to_csv(f'/mnt/c/Users/cesar/Desktop/claro/imports/life_national_life{date}_faltantes_agent_aor.csv', index=False)

# Ameritas

In [4]:
def limpiar_df(df) -> pd.DataFrame:
    df = df.dropna(axis='columns', how='all').dropna( how='all')
    return df

In [13]:
df_ameritas = limpiar_df(pd.read_excel('//mnt/c/Users/cesar/Desktop/claro/life/Ameritas/Ameritas 01-26-2023.xls'))
df_ameritas.head()

,Insured Name,Company Code,Policy Number,Writing Agent Name,Writing Agent ID,Writing Agent Profile,Servicing Agent Name,Servicing Agent ID,Servicing Agent Profile,Product Name,Issue Date,Policy Status
0,"AMADOR, JULLIANNA",100,CI60002150,"ARAQUE, RONNY",AG00105858,1,"ARAQUE, RONNY",AG00105858,1,FLX Living Benefits Index UL Guideline Premium...,04/15/2022,Active/Premium Paying
1,"DIAZ, EDWARD A",100,CI10030745,"RAMOS, WALTER",AG00122703,1,"RAMOS, WALTER",AG00122703,1,FLX Living Benefits Index UL Guideline Premium...,05/18/2021,Active/Premium Paying
2,"GONZALEZ, DANIELA F",100,CI10032287,"TAPIAS, VIRNA",AG00111178,1,"TAPIAS, VIRNA",AG00111178,1,FLX Living Benefits Index UL Guideline Premium...,06/24/2021,Surrendered
3,"MURILLO, RAFAEL A",100,CI10032145,"TAPIAS, VIRNA",AG00111178,1,"TAPIAS, VIRNA",AG00111178,1,FLX Living Benefits Index UL Guideline Premium...,06/24/2021,Active/Premium Paying
4,"MURILLO, VIRNA A",100,CI10033924,"TAPIAS, VIRNA",AG00111178,1,"TAPIAS, VIRNA",AG00111178,1,FLX Living Benefits Index UL Guideline Premium...,10/01/2021,Active/Premium Paying


## Homologación

In [14]:
df_ameritas.columns

Index(['Insured Name', 'Company Code', 'Policy Number', 'Writing Agent Name',
       'Writing Agent ID', 'Writing Agent Profile', 'Servicing Agent Name',
       'Servicing Agent ID', 'Servicing Agent Profile', 'Product Name',
       'Issue Date', 'Policy Status'],
      dtype='object')

In [15]:
df_ameritas = df_ameritas.rename(
    columns={
        'Policy Number':'PolicyNumber',
        'Insured Name':'Insured',
        'Writing Agent Name':'Agent AOR',
        'Policy Status':'Status',
        'Issue Date':'EffectiveDate',
        'Writing Agent ID':'AgentNumber',
        'Product Name':'Product'
        
        
    }
)

### Limpiar Policy Number

In [16]:
# df_a['PolicyNumberClean'] = limpiarPolicyNumber(df_a, 'PolicyNumber')

### Extraer las polizas de National Life

In [21]:
df_b = extractCarrier(df_b, 'Ameritas', 'Carrier')

### Hacer join

In [22]:
def leftJoinBOB(new_policies, bob_policies):
    upload = list(set(new_policies) - set(bob_policies) )
    return upload

In [23]:
upload = leftJoinBOB(df_ameritas['PolicyNumber'].values, df_b['Policy Number'].values)

In [24]:
df_upload = df_ameritas[df_ameritas['PolicyNumber'].isin(upload)]

In [25]:
df_upload

,Insured,Company Code,PolicyNumber,Agent AOR,AgentNumber,Writing Agent Profile,Servicing Agent Name,Servicing Agent ID,Servicing Agent Profile,Product,EffectiveDate,Status
6,"SANCHEZ, JENNIFER",100,CT60009888,"GONZALEZ, ANDRES",AG00110131,2,"GONZALEZ, ANDRES",AG00110131,2,20 Year Term,01/17/2023,Active/Premium Paying


### Upload into template

In [26]:
df_upload.reset_index(drop=True, inplace=True)

In [27]:
df_template = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/Format_Create_BOB.csv').head(0)
df_template

,Carrier___,LOB____,Carrier,Agent AOR,NPN AOR,Contact____,NPN_Internal____,Contact,Policy Number,Source,...,Subsidy,Annual Premium,Paid Thru Date,Days Past Due,Phone Number,Email,Metal Level,Zip Code,Claro Type,Carrier State line of Business


In [28]:
df_template['Member First Name'] = df_upload['Insured'].apply(lambda x : split_names(x, first=False))
df_template['Member Last Name'] = df_upload['Insured'].apply(lambda x : split_names(x))

In [29]:
df_template.reset_index(drop=True, inplace=True)

In [30]:
df_template['Policy Number'] = df_upload['PolicyNumber']

In [31]:
# df_template['Final Premium'] = df_upload['FinalPremium']

In [32]:
df_template['Plan'] = df_upload['Product']

In [33]:
df_template['Agent AOR'] = df_upload['Agent AOR']

In [34]:
df_template['Effective Date'] = df_upload['EffectiveDate']

In [35]:
df_template['LOB____'] = 'Life'

#### Source

In [36]:
def assign_source(referal_agents:list, df_template, column_agent_numbers:str, df_upload) -> pd.DataFrame:
    df_s = df_template.copy()
    mask_s = df_upload[column_agent_numbers].isin(referal_agents)
    
    df_s.loc[mask_s,  'Source'] = 'Referal'
    df_s.loc[~mask_s, 'Source'] = 'Direct'
    
    return df_s

In [37]:
def getSource(query,connection, df_template, df_upload, column_agent_numbers='AgentNumber') -> pd.DataFrame:
    referal_agents = [i[0] for i in query_db(query)]
    referal_list_agents = list(set(referal_agents).intersection(set(df_upload[column_agent_numbers].values)))
    
    df = assign_source(referal_list_agents, df_template, column_agent_numbers, df_upload)
    return df

In [38]:
npn_referals = [18233171, 7388594, 19603471, 18522178, 19106367, 16001347, 10041154, 20462802, 17686050]

npn_referals = str([str(i)  for i in npn_referals]).replace('[', '').replace(']','')


In [39]:
query_referals_agents = """
    SELECT agent_writing_number
    FROM agent_contract 
    WHERE npn in ({list_refarals}) ;
""".format(list_refarals=npn_referals)

In [44]:
df_template = getSource(query_referals_agents, connection, df_template, df_upload)

#### Mermbers

In [45]:
df_template['Members'] = 1

#### Record Type ID

In [46]:
# df_template['Record Type Id']

#### State

In [47]:
df_template['State'] = 'FL'

#### Record Type

In [48]:
def recordType(df_upload, df_template, column_name='Record Type Id', column_agent='AgentNumber'):
    
    record_types = dict(query_db(query_record_type))
    agent_writing_number = query_db(query_awn_jose_m)
    agent_writing_number_jose_m = [i[0] for i in agent_writing_number]
    
#     jose_m_parra_awn_in_df = list(set(agent_writing_number).intersection(set(df_upload[column_agent].values)))
    others_awn = list(set(df_upload[column_agent].values) - set(agent_writing_number_jose_m))
    
    df_r = df_template.copy()
    mask = df_upload[column_agent].isin(others_awn)
    df_r.loc[mask, column_name] = record_types['BOB']
    df_r.loc[~mask, column_name] = record_types['Direct Sales']
    
    return df_r

In [49]:
query_record_type = """
    SELECT record_type, id  FROM record_type rt ;
"""

In [50]:
query_awn_jose_m = """
    SELECT agent_writing_number
    FROM agent_contract 
    WHERE npn = '17795437'
    ;
"""

In [51]:
df_template = recordType(df_upload, df_template)

#### Carrier

In [52]:
query_carrier_id = """
    SELECT carrier_id  
    FROM carrier c 
    WHERE carrier = '{carrier_name}' and state_c = '{carrier_state}'
    ;
""".format(carrier_name= df_b['Carrier'].unique()[0], carrier_state=df_template['State'].unique()[0])

carrier_id = query_db(query_carrier_id)[0][0]

df_template['Carrier'] =  carrier_id

#### Status

In [53]:
df_status = pd.read_excel('/mnt/c/Users/cesar/Desktop/claro/life/Lista Status Life.xlsx')
df_status.head(3)

,Status Carrier,Status SF
0,Active,Active
1,APPROVED,Approved
2,Approved/Admin Review,Approved


In [54]:
df_template['Status'] = df_upload['Status'].apply(lambda x :replace_status(x) )

#### Policy Type

In [55]:
def compare_years_policy(x): 
    one_year_ago = datetime.now() - timedelta(days=365)
    if one_year_ago >= x:
        return 'Renewal'
    return 'New Sale'

In [56]:
df_template['Policy Type'] = pd.to_datetime(df_upload['EffectiveDate']).agg(compare_years_policy)

#### Claro Type

In [57]:
df_template['Claro Type'] = 'New Sale'

#### Last Status

In [58]:
df_template['Last Status Modification Date'] = date

#### Loading Date

In [59]:
df_template['Loading Date'] = date

#### NPN AOR

In [60]:
query_agent_AOR = """
    SELECT contact, npn, agent_writing_number, id
    FROM agent_contract
    ;
"""

In [61]:
def extract_agent_aor(x, df_contact, column_name='NPN'):
    result = df_contact[df_contact['AgentWritingNumber'] == x][column_name].unique()
    assert len(result) < 2, 'Error Data Duplicada en el campo de la base de datos NPN'
    
    if len(result) > 0:
        return result[0]
    return None

In [62]:
def npnAOR(query_agent_AOR,connection, agent_number_colunm_name='AgentNumber'):
    list_contact = query_db(query_agent_AOR)
    df_contact = pd.DataFrame([{'Contact Name':i[0], 'NPN':i[1], 'AgentWritingNumber':i[2], 'Contact ID':i[3]} for i in list_contact])
    
    
    df_template['NPN AOR'] = df_upload[agent_number_colunm_name].apply(lambda x : extract_agent_aor(x, df_contact))
    df_template['Contact'] = df_upload[agent_number_colunm_name].apply(lambda x : extract_agent_aor(x, df_contact, 'Contact ID'))
    return df_template, df_contact

In [63]:
df_template, df_contact = npnAOR(query_agent_AOR, connection)

In [64]:
df_upload['AgentNumber']

0    AG00110131
Name: AgentNumber, dtype: object

#### Contact ID

In [65]:
def extract_contact_id(x, df_contact):
    result = df_contact[df_contact['AgentWritingNumber'] == x]['Contact ID'].unique()
    assert len(result) < 2, 'Error Data Duplicada en el campo de la base de datos NPN'
    
    if len(result) > 0:
        return result[0]
    return None

In [66]:
# df_template['Cotact ID'] = df_upload['AgentNumber'].apply(lambda x : extract_contact_id(x, df_contact))

#### Agent Name

In [67]:
df_template['Agent Name'] = df_upload['Agent AOR']

#### Carrier State Line of Business

In [68]:
query_cslb = """
    select carrier_lob_id 
    from carrier c
    where carrier_id = '{carrier_id}' and state_c = '{state}';
"""

In [69]:
def making_query_cslb(df_template, query_cslb):
    carrier = df_template['Carrier'].unique()
    state = df_template.State.unique()
    
    assert (len(carrier) == 1) and (len(state) == 1), 'Error en la integridad de los datos.'
    
    query_cslb = query_cslb.format(carrier_id=carrier[0], state=state[0])
    value = query_db(query_cslb)[0][0]
    return value

In [70]:
def extract_cslb(df_template, query_cslb):
    cslb = making_query_cslb(df_template, query_cslb)
    df_template['Carrier State line of Business'] = cslb
    
    return df_template

In [71]:
df_template = extract_cslb(df_template, query_cslb)

### Faltantes

In [73]:
df_faltantes = df_template[df_template['Contact'].isna()]

In [75]:
df_template

,Carrier___,LOB____,Carrier,Agent AOR,NPN AOR,Contact____,NPN_Internal____,Contact,Policy Number,Source,...,Annual Premium,Paid Thru Date,Days Past Due,Phone Number,Email,Metal Level,Zip Code,Claro Type,Carrier State line of Business,Agent Name
0,NaN,Life,a0S1G000006iXuWUAU,"GONZALEZ, ANDRES",18233171,NaN,NaN,0031G00000tqyDfQAI,CT60009888,Referal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New Sale,a091G00000B5DUcQAN,"GONZALEZ, ANDRES"


#### Ajustar faltantes

In [1044]:
#df_template = iterpolate(df_template, 'CASTANO PADILLA, FATIMA',  ('18956578','0031G00001JkjjCQAR'))

In [1048]:
df_template

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name
0,"CASTANO PADILLA, FATIMA",18956578,0031G00001JkjjCQAR,a0S1G000006iXuWUAU,FL,CI60006169,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUcQAN,Direct,None,09/16/2022,FLX Living Benefits Index UL Guideline Premium...,1,06-01-2023,06-01-2023,NaN,JULIO,SANTOS
1,"CASTANO PADILLA, FATIMA",18956578,0031G00001JkjjCQAR,a0S1G000006iXuWUAU,FL,CI60006528,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUcQAN,Direct,Terminated,10/15/2022,FLX Living Benefits Index UL Guideline Premium...,1,06-01-2023,06-01-2023,NaN,MEDINA JORGE R,TORRES


## Load

In [76]:
df_bk = df_template.copy()

In [77]:
df_template = df_template[['Agent AOR','NPN AOR', 'Contact', 'Carrier', 'State','Policy Number', 'Policy Type', 'Claro Type','Record Type Id', 'Carrier State line of Business','Source', 'Status', 'Effective Date',
            'Plan','Members', 'Last Status Modification Date', 'Loading Date',  'Final Premium','Member First Name', 'Member Last Name'
            ]]

In [78]:
df_times = df_template.copy()
df_times[['Last Status Modification Date', 'Loading Date']] = df_template[['Last Status Modification Date', 'Loading Date']].apply(lambda x : pd.to_datetime(x, format='%d-%m-%Y').dt.strftime('%m/%d/%Y'), axis='columns')
df_template = df_times

In [86]:
df_template.to_csv(f'/mnt/c/Users/cesar/Desktop/claro/imports/life_ameritas{date}_.csv', index=False)

In [85]:
df_template

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name
0,"GONZALEZ, ANDRES",18233171,0031G00000tqyDfQAI,a0S1G000006iXuWUAU,FL,CT60009888,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUcQAN,Referal,Active,01/17/2023,20 Year Term,1,01/30/2023,01/30/2023,NaN,JENNIFER,SANCHEZ


In [84]:
df_template['Status'] = 'Active'

# America National

In [115]:
df_a = (pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/life/AmricaNational/OR_New America National 01-26-2023.csv')
                     .dropna(axis='columns', how='all')
                     .dropna( how='all'))

In [116]:
df_a.head()

,Agent Name,Policy Type,Policy Number,Critical,Last Update,Product,Annual Premium,Face Amount,State,Applicant,Status,NIA,Requirement,Mailed,Allotment
0,ANDRES FELIPE GONZALEZ,Life New Business,E0247216,No,01/25/2023,Signature Term - 20 Yr,$508.08,"$500,000.00",FL,Sonia Ramirez,Waiting for Requirements,No,Yes,Yes,No
1,ANDRES FELIPE GONZALEZ,Life New Business,E0247201,No,01/16/2023,Signature Term - 20 Yr,$648.00,"$500,000.00",FL,Juan Rojas,Waiting for Requirements,No,Yes,Yes,No
2,DORILUZ PINA,Life New Business,E0247252,No,01/26/2023,Signature Term - 20 Yr,"$1,024.92","$150,000.00",FL,Santos Mendoza,Requirements Received - Pending UW Review,No,No,Yes,No
3,OLVANDA ALTAGRACIA RAMIREZ HERRERA,Life New Business,E0247277,No,01/12/2023,Signature Term - 20 Yr,"$2,553.12","$250,000.00",FL,Gerardo Nunez,Waiting for Requirements,No,Yes,Yes,No
4,OLVANDA ALTAGRACIA RAMIREZ HERRERA,Life New Business,E0247238,No,01/19/2023,Signature Term - 20 Yr,"$5,804.04","$600,000.00",FL,Gerardo Nunez,Waiting for Requirements,No,Yes,Yes,No


## Homologación

In [117]:
df_a.columns

Index(['Agent Name', 'Policy Type', 'Policy Number', 'Critical', 'Last Update',
       'Product', 'Annual Premium', 'Face Amount', 'State', 'Applicant',
       'Status', 'NIA', 'Requirement', 'Mailed', 'Allotment'],
      dtype='object')

In [118]:
df_a = df_a.rename(
    columns={
        'Policy Number':'PolicyNumber',
        'Applicant':'Insured',
        'Agent Name':'Agent AOR',
        'Status':'Status',
        'Last Update':'EffectiveDate',
        'Annual Premium': 'FinalPremium'        
        
    }
)

### Extraer las polizas de National Life

In [119]:
df_b = extractCarrier(df_b, 'American National', 'Carrier')

In [120]:
df_b.head(3)

,Book of Business: BOB Name,Account Executive Name,Carrier,NPN AOR,Agent AOR,Internal Agent,NPN Internal Agent,Contact,Policy Number,Source,...,Email,Metal Level,Zip Code,Book of Business: Last Modified Date,Note,Claro Type,Previous BOB ID,Carrier State line of Business,Book of Business: ID,Account Executive
24550,BOB# 00024753,Luigi Fuentes,American National,8429926.0,INSURANCE XL INC,Adriana Gonzalez,8429926.0,Adriana Gonzalez,UE095096,Direct,...,NaN,NaN,NaN,1/2/2023,NaN,Renewals,NaN,American National Florida,a1C1G000006ZFFH,
24551,BOB# 00024754,,American National,8429926.0,INSURANCE XL INC,Adriana Gonzalez,8429926.0,Adriana Gonzalez,E0125027,Direct,...,NaN,NaN,NaN,1/2/2023,NaN,Renewals,NaN,American National Florida,a1C1G000006ZFFI,
24552,BOB# 00024755,Luigi Fuentes,American National,8429926.0,INSURANCE XL INC,Adriana Gonzalez,8429926.0,Adriana Gonzalez,UE095094,Direct,...,NaN,NaN,NaN,1/2/2023,NaN,Renewals,NaN,American National Florida,a1C1G000006ZFFJ,


### Hacer join

In [121]:
def leftJoinBOB(new_policies, bob_policies):
    upload = list(set(new_policies) - set(bob_policies) )
    return upload

In [122]:
upload = leftJoinBOB(df_a['PolicyNumber'].values, df_b['Policy Number'].values)

In [123]:
df_upload = df_a[df_a['PolicyNumber'].isin(upload)]

In [124]:
df_upload.shape

(0, 15)

In [125]:
df_upload

,Agent AOR,Policy Type,PolicyNumber,Critical,EffectiveDate,Product,FinalPremium,Face Amount,State,Insured,Status,NIA,Requirement,Mailed,Allotment


### Upload into template

In [99]:
df_upload.reset_index(drop=True, inplace=True)

In [100]:
df_template = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/Format_Create_BOB.csv').head(0)

In [101]:
df_template

,Carrier___,LOB____,Carrier,Agent AOR,NPN AOR,Contact____,NPN_Internal____,Contact,Policy Number,Source,...,Subsidy,Annual Premium,Paid Thru Date,Days Past Due,Phone Number,Email,Metal Level,Zip Code,Claro Type,Carrier State line of Business


In [102]:
df_upload

,Agent AOR,Policy Type,PolicyNumber,Critical,EffectiveDate,Product,FinalPremium,Face Amount,State,Insured,Status,NIA,Requirement,Mailed,Allotment


In [103]:
df_template['Member First Name'] = df_upload['Insured'].apply(lambda x : split_names(x, first=False))

In [104]:
df_template['Member Last Name'] = df_upload['Insured'].apply(lambda x : split_names(x))

In [105]:
df_template.reset_index(drop=True, inplace=True)

In [106]:
df_template['Policy Number'] = df_upload['PolicyNumber']

In [107]:
df_template['Final Premium'] = df_upload['FinalPremium']

In [108]:
df_template['Plan'] = df_upload['Product']

In [109]:
df_template['Agent AOR'] = df_upload['Agent AOR']

In [110]:
df_template['Effective Date'] = df_upload['EffectiveDate']

In [111]:
df_template['LOB____'] = 'Life'

In [112]:
#df_an = df_upload.copy()

#df_an['AgentNumber'] = 'Q01P6'

In [113]:
df_upload = df_an

NameError: name 'df_an' is not defined

#### Source

In [210]:
df_template = getSource(query_referals_agents, connection, df_template, df_upload)

#### Mermbers

In [211]:
df_template['Members'] = 1

#### Record Type ID

In [212]:
# df_template['Record Type Id']

#### State

In [213]:
df_template['State'] = 'FL'

#### Record Type

In [214]:
df_template = recordType(df_upload, df_template)

#### Carrier

In [215]:
query_carrier_id = """
    SELECT carrier_id  
    FROM carrier c 
    WHERE carrier = '{carrier_name}' and state_c = '{carrier_state}'
    ;
""".format(carrier_name= df_b['Carrier'].unique()[0], carrier_state=df_template['State'].unique()[0])

carrier_id = query_db(query_carrier_id)[0][0]

df_template['Carrier'] =  carrier_id

#### Status

In [216]:
df_template['Status'] = df_upload['Status'].apply(lambda x :replace_status(x) )

#### Policy Type

In [217]:
df_template['Policy Type'] = pd.to_datetime(df_upload['EffectiveDate']).agg(compare_years_policy)

#### Claro Type

In [218]:
df_template['Claro Type'] = 'New Sale'

#### Last Status

In [219]:
df_template['Last Status Modification Date'] = date

#### Loading Date

In [220]:
df_template['Loading Date'] = date

#### NPN AOR

In [244]:
df_template, df_contact = npnAOR(query_agent_AOR, connection)

In [245]:
df_template

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name
0,LUISA ALVAREZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0238164,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Active,12/27/2022,Signature Term - 30 Yr,1,01/16/2023,01/16/2023,1902.48,Cepeda,Vicky
1,ANDRES FELIPE GONZALEZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247216,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/04/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,508.08,Ramirez,Sonia
2,ANDRES FELIPE GONZALEZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247201,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/06/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,648.00,Rojas,Juan
3,ANGELA P OROZCO,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0242129,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Active,12/22/2022,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,269.52,Villegas,Carolina
4,DORILUZ PINA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247252,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/11/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,1024.92,Mendoza,Santos
5,OLVANDA ALTAGRACIA RAMIREZ HERRERA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247277,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/09/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,2553.12,Nunez,Gerardo
6,OLVANDA ALTAGRACIA RAMIREZ HERRERA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247238,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,12/07/2022,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,5804.04,Nunez,Gerardo


#### Contact ID

In [222]:
# df_template['Contact ID'] = df_upload['AgentNumber'].apply(lambda x : extract_contact_id(x, df_contact))

#### Agent Name

In [223]:
df_template['Agent Name'] = df_upload['Agent AOR']

In [224]:
df_template.dropna(axis='columns', how='all').to_excel(f'/mnt/c/Users/cesar/Desktop/claro/test_life_{date}_.xlsx', index=False)

#### Carrier State Line of Business

In [225]:
df_template = extract_cslb(df_template, query_cslb)

### Check faltante

In [234]:
df_faltantes = df_template[df_template['Contact'].isna()]

In [235]:
df_faltantes

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name


## Interpolacion

In [236]:
#df_template = iterpolate(df_template, 'RAMIREZ SUQUINO DUNIA',  ('20544688','0031G00001YIab5QAD'))

## Load

In [226]:
df_template = df_template[['Agent AOR','NPN AOR', 'Contact', 'Carrier', 'State','Policy Number', 'Policy Type', 'Claro Type','Record Type Id', 'Carrier State line of Business','Source', 'Status', 'Effective Date',
            'Plan','Members', 'Last Status Modification Date', 'Loading Date', 'Final Premium','Member First Name', 'Member Last Name'
            ]]

In [227]:
df_times = df_template.copy()
df_times[['Last Status Modification Date', 'Loading Date']] = df_template[['Last Status Modification Date', 'Loading Date']].apply(lambda x : pd.to_datetime(x, format='%d-%m-%Y').dt.strftime('%m/%d/%Y'), axis='columns')
df_template = df_times

In [230]:
df_template['Final Premium'] = df_template['Final Premium'].apply(lambda x : x.replace('$','').replace(',',''))

In [239]:
df_template

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name
0,LUISA ALVAREZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0238164,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Active,12/27/2022,Signature Term - 30 Yr,1,01/16/2023,01/16/2023,1902.48,Cepeda,Vicky
1,ANDRES FELIPE GONZALEZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247216,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/04/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,508.08,Ramirez,Sonia
2,ANDRES FELIPE GONZALEZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247201,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/06/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,648.00,Rojas,Juan
3,ANGELA P OROZCO,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0242129,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Active,12/22/2022,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,269.52,Villegas,Carolina
4,DORILUZ PINA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247252,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/11/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,1024.92,Mendoza,Santos
5,OLVANDA ALTAGRACIA RAMIREZ HERRERA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247277,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/09/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,2553.12,Nunez,Gerardo
6,OLVANDA ALTAGRACIA RAMIREZ HERRERA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247238,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,12/07/2022,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,5804.04,Nunez,Gerardo


In [238]:
df_template.to_csv(f'/mnt/c/Users/cesar/Desktop/claro/imports/life_america_national_{date}_.csv', index=False)

# Transamerica

In [129]:
df_a = (pd.read_excel('//mnt/c/Users/cesar/Desktop/claro/life/transamerica/TransamericaBook1.xlsx')
                     .dropna(axis='columns', how='all')
                     .dropna( how='all'))

In [130]:
df_a.head()

,ACTIVITY DATE,POLICY NUMBER,INSURED NAME,PRODUCTION CREDIT,WHOLE APP COUNT,PRODUCTION CREDIT.1,WHOLE APP COUNT.1,ANNUAL TARGET PREMIUM,PRODUCT LINE,PRODUCT TYPE
0,12/23/2022,6601437831,LOUISDORT SHERLINE,0.0,0,811.0,1,811.0,Life,Equity Index Universal Life
1,09/27/2022,6601437831,LOUISDORT SHERLINE,811.0,1,0.0,0,811.0,Life,Equity Index Universal Life


## Homologación

In [131]:
df_a.columns

Index(['ACTIVITY DATE', 'POLICY NUMBER', 'INSURED NAME', 'PRODUCTION CREDIT',
       'WHOLE APP COUNT', 'PRODUCTION CREDIT.1', 'WHOLE APP COUNT.1',
       'ANNUAL TARGET PREMIUM', 'PRODUCT LINE', 'PRODUCT TYPE'],
      dtype='object')

In [132]:
df_a = df_a.rename(
    columns={
        'POLICY NUMBER':'PolicyNumber',
        'INSURED NAME':'Insured',
        'Agent Name':'Agent AOR',
        'Status':'Status',
        'ACTIVITY DATE':'EffectiveDate',
        'ANNUAL TARGET PREMIUM': 'FinalPremium'        
        
    }
)

### Extraer las polizas de National Life

In [119]:
df_b = extractCarrier(df_b, 'American National', 'Carrier')

In [120]:
df_b.head(3)

,Book of Business: BOB Name,Account Executive Name,Carrier,NPN AOR,Agent AOR,Internal Agent,NPN Internal Agent,Contact,Policy Number,Source,...,Email,Metal Level,Zip Code,Book of Business: Last Modified Date,Note,Claro Type,Previous BOB ID,Carrier State line of Business,Book of Business: ID,Account Executive
24550,BOB# 00024753,Luigi Fuentes,American National,8429926.0,INSURANCE XL INC,Adriana Gonzalez,8429926.0,Adriana Gonzalez,UE095096,Direct,...,NaN,NaN,NaN,1/2/2023,NaN,Renewals,NaN,American National Florida,a1C1G000006ZFFH,
24551,BOB# 00024754,,American National,8429926.0,INSURANCE XL INC,Adriana Gonzalez,8429926.0,Adriana Gonzalez,E0125027,Direct,...,NaN,NaN,NaN,1/2/2023,NaN,Renewals,NaN,American National Florida,a1C1G000006ZFFI,
24552,BOB# 00024755,Luigi Fuentes,American National,8429926.0,INSURANCE XL INC,Adriana Gonzalez,8429926.0,Adriana Gonzalez,UE095094,Direct,...,NaN,NaN,NaN,1/2/2023,NaN,Renewals,NaN,American National Florida,a1C1G000006ZFFJ,


### Hacer join

In [121]:
def leftJoinBOB(new_policies, bob_policies):
    upload = list(set(new_policies) - set(bob_policies) )
    return upload

In [122]:
upload = leftJoinBOB(df_a['PolicyNumber'].values, df_b['Policy Number'].values)

In [123]:
df_upload = df_a[df_a['PolicyNumber'].isin(upload)]

In [124]:
df_upload.shape

(0, 15)

In [125]:
df_upload

,Agent AOR,Policy Type,PolicyNumber,Critical,EffectiveDate,Product,FinalPremium,Face Amount,State,Insured,Status,NIA,Requirement,Mailed,Allotment


### Upload into template

In [99]:
df_upload.reset_index(drop=True, inplace=True)

In [100]:
df_template = pd.read_csv('//mnt/c/Users/cesar/Desktop/claro/Format_Create_BOB.csv').head(0)

In [101]:
df_template

,Carrier___,LOB____,Carrier,Agent AOR,NPN AOR,Contact____,NPN_Internal____,Contact,Policy Number,Source,...,Subsidy,Annual Premium,Paid Thru Date,Days Past Due,Phone Number,Email,Metal Level,Zip Code,Claro Type,Carrier State line of Business


In [102]:
df_upload

,Agent AOR,Policy Type,PolicyNumber,Critical,EffectiveDate,Product,FinalPremium,Face Amount,State,Insured,Status,NIA,Requirement,Mailed,Allotment


In [103]:
df_template['Member First Name'] = df_upload['Insured'].apply(lambda x : split_names(x, first=False))

In [104]:
df_template['Member Last Name'] = df_upload['Insured'].apply(lambda x : split_names(x))

In [105]:
df_template.reset_index(drop=True, inplace=True)

In [106]:
df_template['Policy Number'] = df_upload['PolicyNumber']

In [107]:
df_template['Final Premium'] = df_upload['FinalPremium']

In [108]:
df_template['Plan'] = df_upload['Product']

In [109]:
df_template['Agent AOR'] = df_upload['Agent AOR']

In [110]:
df_template['Effective Date'] = df_upload['EffectiveDate']

In [111]:
df_template['LOB____'] = 'Life'

In [112]:
#df_an = df_upload.copy()

#df_an['AgentNumber'] = 'Q01P6'

In [113]:
df_upload = df_an

NameError: name 'df_an' is not defined

#### Source

In [210]:
df_template = getSource(query_referals_agents, connection, df_template, df_upload)

#### Mermbers

In [211]:
df_template['Members'] = 1

#### Record Type ID

In [212]:
# df_template['Record Type Id']

#### State

In [213]:
df_template['State'] = 'FL'

#### Record Type

In [214]:
df_template = recordType(df_upload, df_template)

#### Carrier

In [215]:
query_carrier_id = """
    SELECT carrier_id  
    FROM carrier c 
    WHERE carrier = '{carrier_name}' and state_c = '{carrier_state}'
    ;
""".format(carrier_name= df_b['Carrier'].unique()[0], carrier_state=df_template['State'].unique()[0])

carrier_id = query_db(query_carrier_id)[0][0]

df_template['Carrier'] =  carrier_id

#### Status

In [216]:
df_template['Status'] = df_upload['Status'].apply(lambda x :replace_status(x) )

#### Policy Type

In [217]:
df_template['Policy Type'] = pd.to_datetime(df_upload['EffectiveDate']).agg(compare_years_policy)

#### Claro Type

In [218]:
df_template['Claro Type'] = 'New Sale'

#### Last Status

In [219]:
df_template['Last Status Modification Date'] = date

#### Loading Date

In [220]:
df_template['Loading Date'] = date

#### NPN AOR

In [244]:
df_template, df_contact = npnAOR(query_agent_AOR, connection)

In [245]:
df_template

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name
0,LUISA ALVAREZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0238164,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Active,12/27/2022,Signature Term - 30 Yr,1,01/16/2023,01/16/2023,1902.48,Cepeda,Vicky
1,ANDRES FELIPE GONZALEZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247216,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/04/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,508.08,Ramirez,Sonia
2,ANDRES FELIPE GONZALEZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247201,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/06/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,648.00,Rojas,Juan
3,ANGELA P OROZCO,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0242129,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Active,12/22/2022,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,269.52,Villegas,Carolina
4,DORILUZ PINA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247252,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/11/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,1024.92,Mendoza,Santos
5,OLVANDA ALTAGRACIA RAMIREZ HERRERA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247277,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/09/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,2553.12,Nunez,Gerardo
6,OLVANDA ALTAGRACIA RAMIREZ HERRERA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247238,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,12/07/2022,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,5804.04,Nunez,Gerardo


#### Contact ID

In [222]:
# df_template['Contact ID'] = df_upload['AgentNumber'].apply(lambda x : extract_contact_id(x, df_contact))

#### Agent Name

In [223]:
df_template['Agent Name'] = df_upload['Agent AOR']

In [224]:
df_template.dropna(axis='columns', how='all').to_excel(f'/mnt/c/Users/cesar/Desktop/claro/test_life_{date}_.xlsx', index=False)

#### Carrier State Line of Business

In [225]:
df_template = extract_cslb(df_template, query_cslb)

### Check faltante

In [234]:
df_faltantes = df_template[df_template['Contact'].isna()]

In [235]:
df_faltantes

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name


## Interpolacion

In [236]:
#df_template = iterpolate(df_template, 'RAMIREZ SUQUINO DUNIA',  ('20544688','0031G00001YIab5QAD'))

## Load

In [226]:
df_template = df_template[['Agent AOR','NPN AOR', 'Contact', 'Carrier', 'State','Policy Number', 'Policy Type', 'Claro Type','Record Type Id', 'Carrier State line of Business','Source', 'Status', 'Effective Date',
            'Plan','Members', 'Last Status Modification Date', 'Loading Date', 'Final Premium','Member First Name', 'Member Last Name'
            ]]

In [227]:
df_times = df_template.copy()
df_times[['Last Status Modification Date', 'Loading Date']] = df_template[['Last Status Modification Date', 'Loading Date']].apply(lambda x : pd.to_datetime(x, format='%d-%m-%Y').dt.strftime('%m/%d/%Y'), axis='columns')
df_template = df_times

In [230]:
df_template['Final Premium'] = df_template['Final Premium'].apply(lambda x : x.replace('$','').replace(',',''))

In [239]:
df_template

,Agent AOR,NPN AOR,Contact,Carrier,State,Policy Number,Policy Type,Claro Type,Record Type Id,Carrier State line of Business,Source,Status,Effective Date,Plan,Members,Last Status Modification Date,Loading Date,Final Premium,Member First Name,Member Last Name
0,LUISA ALVAREZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0238164,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Active,12/27/2022,Signature Term - 30 Yr,1,01/16/2023,01/16/2023,1902.48,Cepeda,Vicky
1,ANDRES FELIPE GONZALEZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247216,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/04/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,508.08,Ramirez,Sonia
2,ANDRES FELIPE GONZALEZ,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247201,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/06/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,648.00,Rojas,Juan
3,ANGELA P OROZCO,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0242129,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Active,12/22/2022,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,269.52,Villegas,Carolina
4,DORILUZ PINA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247252,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/11/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,1024.92,Mendoza,Santos
5,OLVANDA ALTAGRACIA RAMIREZ HERRERA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247277,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,01/09/2023,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,2553.12,Nunez,Gerardo
6,OLVANDA ALTAGRACIA RAMIREZ HERRERA,7733898,0031G00000pFnQwQAK,a0S1G000006ag2kUAA,FL,E0247238,New Sale,New Sale,0121G000000bpwvQAA,a091G00000B5DUYQA3,Direct,Pending,12/07/2022,Signature Term - 20 Yr,1,01/16/2023,01/16/2023,5804.04,Nunez,Gerardo


# Close Connection DB

In [ ]:
connection.close()

# Test

In [18]:
df = pd.read_csv('//mnt/c/Users/cesar/Downloads/verificacion.csv', low_memory=False)

In [7]:
df.columns

Index(['Book of Business: BOB Name', 'Account Executive Name', 'Carrier',
       'NPN AOR', 'Agent AOR', 'Internal Agent', 'NPN Internal Agent',
       'Contact', 'Policy Number', 'Source', 'Referrals Id', 'Status',
       'Effective Date', 'Policy Term Date', 'Member First Name',
       'Member Last Name', 'DOB', 'State', 'Members', 'Policy Type',
       'Automatic Renewal', 'Line of Business',
       'Last Status Modification Date', 'Loading Date',
       'Book of Business: Created Date', 'Book of Business: Record Type',
       'Agency', 'Plan', 'Final Premium Currency', 'Final Premium',
       'Subsidy Currency', 'Subsidy', 'Annual Premium Currency',
       'Annual Premium', 'Renewal Type', 'AEP 2023', 'Renewal 2023',
       'All 2023', 'Account Name', 'Paid Thru Date', 'Days Past Due', 'County',
       'City', 'Phone Number', 'Email', 'Metal Level', 'Zip Code',
       'Book of Business: Last Modified Date', 'Note', 'Claro Type',
       'Previous BOB ID', 'Carrier State line of Busi

In [102]:
df['Effective Date'] = pd.to_datetime(df['Effective Date'], format='%m/%d/%Y')

In [112]:
a = df[df['Effective Date'] <= '1/1/2023']

In [124]:
a = a[a['Status'] == 'Active']

In [130]:
a[a['Line of Business'] == 'Life'].shape

(909, 55)

In [100]:
df[df['Effective Date'] == '1/1/2023']

,Book of Business: BOB Name,Account Executive Name,Carrier,NPN AOR,Agent AOR,Internal Agent,NPN Internal Agent,Contact,Policy Number,Source,...,Metal Level,Zip Code,Book of Business: Last Modified Date,Note,Claro Type,Previous BOB ID,Carrier State line of Business,Book of Business: ID,Account Executive,policies
531,BOB# 00000557,,Oscar,9110006.0,Francisco Serrano,Francisco Emanuel Serrano,9110006.0,Francisco Emanuel Serrano,OSC05950837-01,Direct,...,Silver,NaN,1/2/2023,NaN,Renewals,NaN,Oscar Florida,a1C1G000007eM7h,,1
533,BOB# 00000559,,Oscar,19279907.0,NOMAR SALAS,Nomar Salas,19279907.0,Nomar Salas,OSC04813929-01,Direct,...,Silver,NaN,1/18/2023,NaN,Renewals,NaN,Oscar Florida,a1C1G000007eM7j,,1
583,BOB# 00000609,,Oscar,18294087.0,WANDA IVETTE MATOS - ARROYO,Wanda Ivette Matos,18294087.0,Wanda Ivette Matos,OSC05819800-01,Direct,...,Silver,32825.0,1/11/2023,V-Actualización DtR 10-30,Renewals,NaN,Oscar Florida,a1C1G000007eM8X,,1
615,BOB# 00000641,,Oscar,16001347.0,Gamil Kharfan,Wanda Ivette Matos,18294087.0,Wanda Ivette Matos,OSC05821988-01,Referral,...,Silver,34769.0,1/23/2023,NaN,Renewals,BOB# 00007632,Oscar Florida,a1C1G000007eM93,,1
621,BOB# 00000647,,Oscar,9110006.0,Francisco Serrano,Francisco Emanuel Serrano,9110006.0,Francisco Emanuel Serrano,OSC05932363-01,Direct,...,Silver,NaN,1/11/2023,NaN,Renewals,NaN,Oscar Florida,a1C1G000007eM99,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199689,BOB# 00223103,,United Health Care,18796340.0,"MEZA BARRETO, MARTHA",Martha Meza Barreto,18796340.0,Martha Meza Barreto,12560631501,Direct,...,NaN,NaN,1/24/2023,NaN,New Sale,NaN,United Health Care Texas,a1C1G000009UJ06,,1
199690,BOB# 00223104,,United Health Care,18796340.0,"MEZA BARRETO, MARTHA",Martha Meza Barreto,18796340.0,Martha Meza Barreto,12560631502,Direct,...,NaN,NaN,1/24/2023,NaN,New Sale,NaN,United Health Care Texas,a1C1G000009UJ07,,1
199736,BOB# 00223171,,United Health Care,451611.0,"ALPIZAR, ODALYS",Odalys Alpizar,451611.0,Odalys Alpizar,12442940601,Direct,...,NaN,NaN,1/24/2023,NaN,New Sale,NaN,United Health Care Florida,a1C1G000009UJ1C,,1
199737,BOB# 00223172,,United Health Care,451611.0,"ALPIZAR, ODALYS",Odalys Alpizar,451611.0,Odalys Alpizar,12442940602,Direct,...,NaN,NaN,1/24/2023,NaN,New Sale,NaN,United Health Care Florida,a1C1G000009UJ1D,,1


In [15]:
df[(df['Status'] == 'Approved')&(df['Line of Business'] == 'Life')].shape

(5, 54)

In [52]:
df[(df['Carrier'] == 'Senior Life') & (df['State'] == 'Florida')]['Annual Premium'].shape

(4,)

In [69]:
round(df[(df['Carrier'] == 'National Life') & (df['State'] == 'Texas') &(df['Line of Business'] == 'Life')]['Annual Premium'].sum(), 2)

45643.04

In [78]:
df[(df['State'] == 'Tennessee') &(df['Line of Business'] == 'Life')].shape

(1, 54)

In [84]:
df['policies'] = 1

In [96]:
df[df['Line of Business'] == 'Life'].groupby('Agency').count()[['policies']].sort_values(by='policies', ascending=False).head(50)

,policies
Agency,
Prestige Insurance Consultant,355
Elite Insurance Agents Corp,193
Universal Insurance agency 360 Corp,127
Advisors 4LIfe,101
Claro Insurance,100
Janeth Isea,49
Florida Global Insurance,42
Comfort Insurance,40
Solutions Advisor Insurance INC,32
